# DVCS

Connect traits to distributed version control systems like `git` and `fossil` to capture changes and metadata over time.

> _The initial implementation will be mostly served by the kernel, but a browser-side implementation seems possible_

## Wanted: Simple Example

It should be easy to link a single widget to a sane on-disk representation. 

```python
import ipywidgets as W
from wxyz.dvcs import Repo, WidgetWatcher

text = W.Textarea()
repo = Repo()
repo.watch(text, ["value"], "value.json")
```

## Concepts

### Repo
- Low-level representation of a repo.
- Can list 
  - `Head`
  - `Remote`

### Remote
A remote repo that can be 
- can list
  - `Refs`
- (at least) `fetch`
- (maybe) `push`ed to


### Head
- a named commit
- can list `RepoFile`

### RepoFile

A file in the repo
- can get the value, commi

### Diff

A diff of two refs

## Prototype fiddling

In [ ]:
import ipywidgets as W, traitlets as T, git as G, pathlib as P, time, pprint, ruamel_yaml
from datetime import datetime

In [ ]:
def make_repo_explorer():
    return W.HBox([
        W.Label("History"),
        W.Select(options=["origin", "collaborator1"], description="remote", multiple=False, rows=1),
        W.Select(options=["main", "dev"], description="head", multiple=False, rows=1)
    ])
make_repo_explorer()

In [ ]:
def make_actions():
    return W.VBox([
        W.Label("Actions"), 
        W.HBox([
            W.Label("Save"), 
            W.HTML("<code>+100/-75</code>", description="Δ"),
            W.Text(description="Message", placeholder="Updated value.yaml"),
            W.Button(description="Save", icon="check")
        ]), 
        W.HBox([
            W.Label("Restore"), 
            W.Select(options=["main", "dev"], description="head", multiple=False, rows=1),
            W.SelectionSlider(options=["just now", "5 minutes ago"], description="history", multiple=False, rows=1),
            W.HTML("<code>+100/-75</code>", description="Δ"),
            W.Button(description="Revert", icon="refresh")
        ]),
    ])
make_actions()

In [ ]:
from wxyz.dvcs import Watcher

In [ ]:
class Commit(W.Widget):
    message = T.Unicode()
    summary = T.Unicode()
    authored = T.Instance(datetime)
    committed = T.Instance(datetime)
    committer = T.Unicode()
    author = T.Unicode()
    paths = W.trait_types.TypedTuple(T.Instance(P.Path))

In [ ]:
class Repo(W.Widget):
    working_dir = T.Instance(P.Path)
    watching = T.Bool(default_value=False)
    dirty = T.Bool(default_value=False)
    changes = T.Tuple(allow_none=True)
    _watcher = T.Instance(Watcher, allow_none=True)
    _change_link = None
    
    def __init__(self, working_dir, *args, **kwargs):
        kwargs["working_dir"] = P.Path(kwargs.get("working_dir", working_dir))
        super().__init__(*args, **kwargs)
    
    @T.default("changes")
    def _default_changes(self):
        return tuple()
    
    @T.observe("watching", "path")
    def _on_watching(self, change):
        if self._watcher:
            if self._change_link:
                self._change_link.unlink()
                self._change_link = None
            self._watcher.watching = False
            self._watcher = None
        if self.watching:
            self._watcher = Watcher(self.working_dir)
            self._change_link = T.dlink((self._watcher, "changes"), (self, "changes"), self._on_watch_changes)
            self._watcher.watching = True
    
    def _on_watch_changes(self, changes):
        """ react to changes from the watcher
        """
        return changes

In [ ]:
class Git(Repo):
    _git = T.Instance(G.Repo, allow_none=True)
    
    @T.observe("working_dir")
    def _on_path(self, change):
        if change.new:
            self._git = G.Repo.init(change.new)
        
    
    def _on_watch_changes(self, changes):
        self.dirty = self._git.is_dirty()
        return [
            dict(
                a_path=diff.a_path,
                b_path=diff.b_path,
                change_type=diff.change_type
            )
            for diff in self._git.index.diff(None)
        ] + [
            dict(
                a_path=None,
                b_path=ut,
                change_type="U"
            ) for ut in repo._git.untracked_files
        ]
    
    def stage(self, path):
        self._git.index.add(path)
    
    def unstage(self, path):
        self._git.index.remove(path)
    
    def commit(self, message):
        self._git.index.commit(message)

# A Demo UI

In [ ]:
changes = W.VBox()
commit_message = W.Textarea(description="message")
path = W.HTML()
watching = W.Checkbox(description="Watching?")
dirty = W.Checkbox(description="Dirty?", disabled=True)
commit = W.Button(description="Commit")

In [ ]:
repo = Git("_foo")

In [ ]:
def make_change(change):
    stage = W.Checkbox(description="Stage")
    def on_stage(self):
        repo.stage(change.b_path or change.a_path)
    stage.observe(on_stage, "value")
    
    unstage = W.Checkbox(description="Unstage")
    def on_unstage(self):
        repo.unstage(change.b_path or change.a_path)
    unstage.observe(on_unstage, "value")
        
    return W.HBox([
        W.HTML(change.b_path or change.a_path),
        stage,
        unstage
    ])

In [ ]:
def change_to_children(changes):
    return list(map(make_change, repo._git.index.diff(None)))

In [ ]:
T.dlink((repo, "working_dir"), (path ,"value"), "<code>{}</code>".format)
T.dlink((repo, "changes"), (changes, "children"), change_to_children)
T.link((repo, "watching"), (watching, "value"))
T.dlink((repo, "dirty"), (dirty, "value"))
commit.on_click(lambda x: repo.commit(commit_message.value))

In [ ]:
head = W.HBox([
    path,
    dirty,
    watching
])
body = W.HBox([
    W.VBox([
        W.HTML("<h2>Uncommitted Changes</h2>"),
        changes
    ]),
    W.VBox([
        W.HTML("<h2>Commit</h2>"),
        commit_message,
        commit
    ])
])
ui = W.VBox([head, body])
ui

In [ ]:
readme = (repo.working_dir / "README.md")
readme.write_text(f"# f00: {time.time()}")

In [ ]:
gitignore = (repo.working_dir / ".gitignore")
gitignore.write_text("""
.ipynb_checkpoints/
""")

In [ ]:
repo._git.index.add([readme.name,  gitignore.name])

In [ ]:
repo._git.index.commit(f"woop {time.time()}")

In [ ]:
diff = repo._git.index.diff(None)[0]

In [ ]:
diff.b_path

In [ ]:
repo._git.index.entries

In [ ]:
repo._git.index.remove([".gitignore"])

In [ ]:
repo._git.index.diff(None)

In [ ]:
!cd _foo && git status

In [ ]:
repo._git.index.entries